# Halley's Comet - Trajectory Prediction

## Contributors
* Ivan Cadiang
* Rohan Solas

## Introduction
The three-dimensional equations of motion of a planetary object around the sun is given by

$$
  \begin{cases}
    \frac{d^{2}x}{dt^{2}}&=-4\pi^{2} \frac{x}{\left(x^{2}+y^{2}+z^{2}\right)^{\frac{3}{2}}} \\
    \frac{d^{2}y}{dt^{2}}&=-4\pi^{2} \frac{y}{\left(x^{2}+y^{2}+z^{2}\right)^{\frac{3}{2}}} \\
    \frac{d^{2}z}{dt^{2}}&=-4\pi^{2} \frac{z}{\left(x^{2}+y^{2}+z^{2}\right)^{\frac{3}{2}}}
  \end{cases}
$$
Halley's comet was seen on its perihelion (or the nearest distance to the sun that becomes visible on Earth) on February 9, 1986. Scientists mapped the position to be
$$
  \begin{bmatrix}
    x \\ y \\ z
  \end{bmatrix}=\begin{bmatrix}
      0.325514\,\text{Au} \\
      -0.459460\,\text{Au} \\
      0.166229\,\text{Au}
  \end{bmatrix}
$$
And its velocity to be

$$
  \begin{bmatrix}
    dx/dt \\ dy/dt \\ dz/dt
  \end{bmatrix}=\begin{bmatrix}
      -0.096111\,\text{Au/year} \\
      -6.916687\,\text{Au/year} \\
      -1.305721\,\text{Au/year}
  \end{bmatrix}
$$

We want to reconstruct the comet's 3D orbit by solving the IVP using RK4 with step-size doubling/halving. We will be solving for the trajectory for 200 years since it was last seen (February 9, 1986).

In [1]:
# Add important libraries for vecotrized computations and plotting
import numpy as np
import plotly.express as px # https://plotly.com/python/3d-scatter-plots/
import pandas as pd # https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/

# Add extra libraries here
import math


## Finding the acceleration


Let $\vec{p}$, $\vec{v}$, and $\vec{a}$ be the comet's position, velocity, and acceleration, respectively.


Given position $\vec{p}$, we calculate its acceleration

In [2]:
def accel(p):
    x = p.item(0) # x-coordinate
    y = p.item(1) # y-coordinate
    z = p.item(2) # z-coordinate

    a = np.empty((3,)) # Initialize acceleration vector

    # Calculate acceleration using the formula:
    r = (x**2 + y**2 + z**2)**(1.5)
    a[0] = -4 * math.pi**2 * x / r
    a[1] = -4 * math.pi**2 * y / r
    a[2] = -4 * math.pi**2 * z / r

    return a # Return acceleration vector

print(accel(np.array([0.325514, -0.459460, 0.166229])))

[-63.50048662  89.63034949 -32.4275527 ]


## Deriving the RK4 System of Equations
From physics, we know that $\dot{p}=\vec{v}$ and $\dot{\vec{v}}=\vec{a}$.

This means that $$\vec{v}_{\vec{p}_{i+1}}=f(t_i+\Delta t,\vec{p}_i)=\vec{a}_{\vec{p}_i}\cdot \Delta t+\vec{v}_{\vec{p}_i}$$
for some small step in time $\Delta t>0$.

Suppose we wish to solve the system using RK4 to estimate the position of the comet, derive the iterative equations for $\vec{v}_{i+1}$ and $\vec{p}_{i+1}$ where $h$ is the current time step.

For $\vec{v}$,
$$
\begin{align*}
  k_{1,v}&= \vec{a}_{p_i} \\
  k_{2,v}&= \vec{a}_{\vec{p}_i+ \frac{h}{2} \cdot k_{1,p}} \\
  k_{3,v}&= \vec{a}_{\vec{p}_i+ \frac{h}{2} \cdot k_{2,p}} \\
  k_{4,v}&= \vec{a}_{\vec{p}_i+ h \cdot k_{3,p}} \\
\end{align*}
$$

And now for $\vec{p}$,

$$
\begin{align*}
  k_{1,p}&= \vec{v}_i \\
  k_{2,p}&= \frac{h}{2} \cdot k_{1,v} + \vec{v}_i\\
  k_{3,p}&= \frac{h}{2} \cdot k_{2,v} + \vec{v}_i\\
  k_{4,p}&= h \cdot k_{3,v} + \vec{v}_i\\
\end{align*}
$$

Now, we build $\vec{v}_{i+1}$ and $\vec{p}_{i+1}$.
$$
\begin{align*}
  \vec{v}_{i+1}&= \vec{v}_i + \dfrac{h}{6} (k_{1,v} + 2k_{2,v} + 2k_{3,v} + k_{4,v}) \\
  \vec{p}_{i+1}&= \vec{p}_i + \dfrac{h}{6} (k_{1,p} + 2k_{2,p} + 2k_{3,p} + k_{4,p})
\end{align*}
$$

Now, we use the system we just derived in a function that calculates one iteration of RK4.

In [3]:
def RK4_single(p, v, h):
  # p_prime = v => y = p, f = y' = v
  # v_prime = a => y = v, f = y' = a

  k_1v = accel(p)
  k_1p = v

  k_2v = accel(p + h/2 * k_1p)
  k_2p = h/2 * k_1v + v

  k_3v = accel(p + h/2 * k_2p)
  k_3p = h/2 * k_2v + v

  k_4v = accel(p + h * k_3p)
  k_4p = h * k_3v + v


  v1 = v + (h/6)*(k_1v + 2*k_2v + 2*k_3v + k_4v)
  p1 = p + (h/6)*(k_1p + 2*k_2p + 2*k_3p + k_4p)

  return v1, p1 # You may modify the return values here if you want
  # to use this method for the next cell

When performing adaptive step size computations, we need **two** sets of values. The first computation is for $\Delta t$ and the other one is for $\Delta t / 2$.

After obtaining `v1` and `p1`, perform another iteration of RK4 with time step $\Delta t/2$.

In [4]:
def RK4_adaptive(p, v, h):
  v1, p1 = RK4_single(p, v, h)
  v2, p2 = RK4_single(p1, v1, h/2)
  v2, p2 = RK4_single(p2, v2, h/2)

  return v1, p1, v2, p2 # v2 and p2 are the values obtained from h/2

## Assembling the Adaptive RK4 algorithm

Now that the coefficients are done, we will now implement the changes in the step size. The following are the parameters that the algorithm will use.

*   Initial time step $h$
*   Minimum step size $h_{min}$ so that the algorithm doesn't stop
*   Minimum and maximum tolerance $\varepsilon_{min}$ and $\varepsilon_{max}$ (inclusive) denotes the bounds wherein the step size doesn't change.

After computing for the coefficients $\texttt{v2}$ and $\texttt{p2}$, we want to assemble our algorithm.

Adaptive step size works by computing for the *largest* absolute error between the computed $p$ values from $\Delta t$ and $\Delta t/2$. This value is a scalar.

If the error is large, we want to keep halving the step size until the error is within the bounds defined. If the error is too small, we save $\Delta t$ and the step size is doubled. Otherwise, we will only save the approximation at $\Delta t$.

In [5]:
def RK4_iterate(p, v, h, t_start, t_end, h_min, e_min, e_max):
  # The inital value is the sun positioned at (0, 0, 0)
  time, x_computed, y_computed, z_computed, distance_computed, step_size_computed = [t_start], [0], [0], [0], [30], [h]
  # Add your code here
  while t_start < t_end:
        # Perform RK4 with constant step size h
        v1, p1, v2, p2 = RK4_adaptive(p, v, h)

        # Compute the largest absolute error between p values
        error = np.max(np.abs(p1 - p2))

        if error > e_max:
            # Halve the step size and redo
            h /= 2
        elif error < e_min and h < h_min:
            # Double next step size and update t_start
            t_start += h

            time.append(t_start)
            step_size_computed.append(h)
            x_computed.append(p1[0])
            y_computed.append(p1[1])
            z_computed.append(p1[2])
            distance_computed.append(np.sqrt(p1[0]**2 + p1[1]**2 + p1[2]**2))

            p = p1
            v = v1

            h *= 2
        else:
            # Save the approximation at t_start
            t_start += h

            # While iterating, append the values to the lists provided above.
            time.append(t_start)
            step_size_computed.append(h)
            x_computed.append(p1[0])
            y_computed.append(p1[1])
            z_computed.append(p1[2])
            distance_computed.append(np.sqrt(p1[0]**2 + p1[1]**2 + p1[2]**2))

            p = p1
            v = v1

  return (time, x_computed, y_computed, z_computed, distance_computed, step_size_computed)

## Generating and plotting the points for the comet's trajectory

In [6]:
# Initial parameters
h = 0.01
t_start = 0.00
t_end = 200.0
h_min = 1e-9
e_min = 1e-3
e_max = 1e-2

p0 = np.array([0.325514, -0.459460, 0.166229])
v0 = np.array([-0.096111, -6.916687, -1.305721])

In [7]:
# NOTE: Takes about a minute and a half to execute
time, xdata, ydata, zdata, distances, step_sizes = RK4_iterate(p0, v0, h, t_start, t_end, h_min, e_min, e_max)

In [11]:
df = pd.DataFrame(dtype = float)
df['x'] = xdata
df['y'] = ydata
df['z'] = zdata
df['time'] = time
df['step_size'] = step_sizes
df['distance_from_sun'] = distances
fig = px.scatter_3d(df, x='x', y='y', z='z', color='distance_from_sun')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

We also have data values for `(time, distance_computed)`. This means that we will plot its distance from the sun over time.

In [ ]:
fig2 = px.scatter(df, x='time', y='distance_from_sun')
fig2.show()

To show that the step size is indeed adaptive, we plot it over time.

In [ ]:
fig2 = px.line(df, x='time', y='step_size')
fig2.show()